# Convolutional Neural Network

### Importing the libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [3]:
tf.__version__

'2.4.0'

In [4]:
!wget --no-check-certificate \
    https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-Convolutional-Neural-Networks.zip \
    -O /tmp/cat-or-dog.zip

--2020-12-30 09:24:32--  https://sds-platform-private.s3-us-east-2.amazonaws.com/uploads/P16-Convolutional-Neural-Networks.zip
Resolving sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)... 52.219.101.178
Connecting to sds-platform-private.s3-us-east-2.amazonaws.com (sds-platform-private.s3-us-east-2.amazonaws.com)|52.219.101.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 234338911 (223M) [application/zip]
Saving to: ‘/tmp/cat-or-dog.zip’

/tmp/cat-or-dog.zip 100%[===================>] 223.48M  44.6MB/s    in 5.5s    

2020-12-30 09:24:37 (40.9 MB/s) - ‘/tmp/cat-or-dog.zip’ saved [234338911/234338911]



In [5]:
import os
import zipfile

local_zip = '/tmp/cat-or-dog.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cat-or-dog')
zip_ref.close()

In [8]:
# Directory with our training dog pictures
train_dog_dir = os.path.join('/tmp/cat-or-dog/Part 2 - Convolutional Neural Networks/dataset/training_set/dogs')

# Directory with our training cat pictures
train_cat_dir = os.path.join('/tmp/cat-or-dog/Part 2 - Convolutional Neural Networks/dataset/training_set/cats')
print(train_cat_dir)

/tmp/cat-or-dog/Part 2 - Convolutional Neural Networks/dataset/training_set/cats


In [10]:
print('total training dog images:', len(os.listdir(train_dog_dir)))
print('total training cat images:', len(os.listdir(train_cat_dir)))
print('total training images',len(os.listdir(train_dog_dir))+len(os.listdir(train_cat_dir)) )

total training dog images: 4001
total training cat images: 4001
total training images 8002


## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/tmp/cat-or-dog/Part 2 - Convolutional Neural Networks/dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [12]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/tmp/cat-or-dog/Part 2 - Convolutional Neural Networks/dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [13]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [14]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [15]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [17]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [18]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [19]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [20]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [21]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 64s 252ms/step - loss: 0.6692 - accuracy: 0.5824 - val_loss: 0.5924 - val_accuracy: 0.6965
Epoch 2/25
250/250 [==============================] - 62s 249ms/step - loss: 0.5963 - accuracy: 0.6703 - val_loss: 0.5457 - val_accuracy: 0.7245
Epoch 3/25
250/250 [==============================] - 62s 249ms/step - loss: 0.5413 - accuracy: 0.7304 - val_loss: 0.5332 - val_accuracy: 0.7390
Epoch 4/25
250/250 [==============================] - 62s 248ms/step - loss: 0.5233 - accuracy: 0.7327 - val_loss: 0.5173 - val_accuracy: 0.7535
Epoch 5/25
250/250 [==============================] - 62s 248ms/step - loss: 0.4904 - accuracy: 0.7631 - val_loss: 0.5177 - val_accuracy: 0.7500
Epoch 6/25
250/250 [==============================] - 62s 249ms/step - loss: 0.4719 - accuracy: 0.7727 - val_loss: 0.4644 - val_accuracy: 0.7740
Epoch 7/25
250/250 [==============================] - 62s 248ms/step - loss: 0.4558 - accuracy: 0.7802 - val_loss: 0.4787 - val_ac

## Part 4 - Making a single prediction

In [27]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/tmp/cat-or-dog/Part 2 - Convolutional Neural Networks/dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

In [28]:
print(prediction)

dog


In [29]:
print(result)

[[1.]]
